In [14]:
from pydantic import BaseModel
import pandas as pd
import os
import json
import utils
import time
from openai import OpenAI

In [15]:
with open("./apikey_openai.txt", "r") as file:
    apikey_openai = file.read().strip()
    os.environ["OPENAI_API_KEY"] = apikey_openai

with open("./instruction_toLLM/extract_scandal_company_name.txt", "r") as file:
    instruction = file.read().strip()

In [16]:
class scandalous_coumpany(BaseModel):
    no: int
    s_c: str
    n: str
    nn: str

class Inspectation(BaseModel):
    final_result: list[scandalous_coumpany]

In [17]:
scandal_df = pd.read_csv("./data/scandals_20200101to20241113.csv")
done_df = pd.read_csv("./data/done.csv")

scandal_df

,title,date,url
0,損保ジャパン「不祥事忘れない」 社員向けに資料展示室,2024-11-07 15:05:18,https://www.nikkei.com/article/DGXZQOUB070900X...
1,一大イベントの直前に不祥事発覚 カナデビア社長「極めて残念」,2024-11-08 15:02:29,https://www.asahi.com/articles/ASSC81SZQSC8ULF...
2,不祥事相次ぐ鹿児島県警 警察庁の見解は? 書面で回答された本部長の転出経緯と、「隠蔽の疑惑な...,2024-11-09 07:00:00,https://373news.com/_news/storyid/204175/
3,また土木部...25歳福島県職員が逮捕 盗撮か 入札巡る県職員3人の起訴に続き 再発防止の徹...,2024-11-06 19:20:00,https://www.fukushima-tv.co.jp/localnews/2024/...
4,ＩＣＣ主任検察官の不祥事調査,2024-11-12 06:36:26,https://webun.jp/articles/-/706439
...,...,...,...
15256,MOMOLAND デイジー、インタビューで番組の不正を暴露…所属事務所＆CN ENMは反論「...,2020-01-07 17:00:00,https://news.kstyle.com/article.ksn?articleNo\...
15257,【国際免許証、偽造も】外国人レンタカー利用者100万人突破！ 「不正な」国際免許証に要注意,2020-01-03 17:00:00,https://www.autocar.jp/post/453984
15258,元会長カルロス・ゴーン氏のレバノン逃亡について日産が声明を発表。その内容とは,2020-01-07 17:00:00,https://driver-web.jp/articles/detail/26573
15259,KinKi Kids・堂本光一、電子チケット転売に不快感！ 「入場後に席替えてもらうバカ」と...,2020-01-08 17:00:00,https://cyzowoman.jp/2020/01/post_264998_1.html


In [18]:
scandal_df = scandal_df[~scandal_df["title"].isin(done_df["title"])]
scandal_df = scandal_df.reset_index(drop=True)
scandal_df

,title,date,url
0,日野と三菱ふそうが経営統合へ向かう2つの要因 親会社のトヨタとダイムラーがCASE技術で協業,2023-06-03 05:51:35,https://toyokeizai.net/articles/-/676950
1,「メガネスーパー」、生活保護受給者向け販売を全店中止,2023-06-02 16:00:00,https://www.nikkei.com/article/DGXZQOUC02B210S...
2,かんぽ不正巡る解雇、札幌地裁「相当」 同種訴訟で判断分かれる [北海道],2023-06-05 16:00:00,https://www.asahi.com/articles/ASR65635VR65IIP...
3,＜２０２３年６月１日付組織改正＞認証申請での不正行為を受け、開発と法規認証の体制を見直し,2023-05-31 16:00:00,https://prtimes.jp/main/html/rd/p/000000307.00...
4,北海道運輸局、千歳相互観光バスを調査 部品、不正交換の疑い [北海道],2023-05-31 16:00:00,https://www.asahi.com/articles/ASR5056NJR50IIP...
...,...,...,...
6485,MOMOLAND デイジー、インタビューで番組の不正を暴露…所属事務所＆CN ENMは反論「...,2020-01-07 17:00:00,https://news.kstyle.com/article.ksn?articleNo\...
6486,【国際免許証、偽造も】外国人レンタカー利用者100万人突破！ 「不正な」国際免許証に要注意,2020-01-03 17:00:00,https://www.autocar.jp/post/453984
6487,元会長カルロス・ゴーン氏のレバノン逃亡について日産が声明を発表。その内容とは,2020-01-07 17:00:00,https://driver-web.jp/articles/detail/26573
6488,KinKi Kids・堂本光一、電子チケット転売に不快感！ 「入場後に席替えてもらうバカ」と...,2020-01-08 17:00:00,https://cyzowoman.jp/2020/01/post_264998_1.html


In [19]:
start_time = time.time()

article_ls = scandal_df["title"].to_list()
article_numbered_ls = [f"{i}.{item}" for i, item in enumerate(article_ls)]

batch_size = 100


try:
    for i in range(0, len(article_ls), batch_size):
        batch_ls = article_numbered_ls[i:i+batch_size]

        result_raw = utils.call_openai_structured_output(
                instruction= instruction,
                prompt= str(batch_ls),
                model = "gpt-4o",
                response_format= Inspectation
        )

        result_inspection = result_raw.final_result

        for item in result_inspection:
            row_index = item.no
            scandal_df.loc[row_index, 'scandal_of_company'] = item.s_c
            scandal_df.loc[row_index, 'company_name'] = item.n
            scandal_df.loc[row_index, 'related_company_name'] = item.nn

        

        # result_str = utils.call_openai_basic(instruction, prompt=str(batch_ls), model="gpt-4o")
        # result_str = result_str.content

        # chatGPTの表記揺れの修正
        # if result_str.startswith("```json"):
        #     result_str = result_str[7:-3]

        # if result_str.startswith("\n"):
        #     result_str = result_str[1:]
        
        # if result_str.startswith("\n"):
        #     result_str = result_str[1:]

        # if result_str.endswith("\n`"):
        #     result_str = result_str[:-2]

        #result_json = json.loads(result_str)

        # for item in result_json:
        #     row_index = int(item['no']) 
        #     scandal_df.loc[row_index, 'scandal_of_company'] = item['s_c']
        #     scandal_df.loc[row_index, 'company_name'] = item['n']
        #     scandal_df.loc[row_index, 'related_company_name'] = item['nn']

        rap = time.time()
        elapsed_time = rap - start_time
        print(f"{i} 回目のバッチ終了！：{elapsed_time:.2f} s")

        start_time = time.time()

        1 + "a"


except Exception as e:
    scandal_df.to_csv("scandal_df_partial_output.csv", index=False)
        # 処理時間を測定
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Error was captured!!!")
    print(e)


scandal_df.to_csv("./data/scandals_20200101to20241113_symbol_added.csv", index=False)

# 処理時間を測定
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Processing time: {elapsed_time:.2f} seconds")





0 回目のバッチ終了！：48.25 s
Error was captured!!!
unsupported operand type(s) for +: 'int' and 'str'
Processing time: 0.04 seconds
